In [1]:
from src.pricing.base.rate import Rate
from src.pricing.base.volatility import Volatility
from src.utility.types import Maturity

In [2]:
from src.pricing.structured_products import ReverseConvertible


rcki = ReverseConvertible(rate= Rate(0.03),
        maturity= Maturity(1),
        nominal=100,
        spot_price=100,
        strike_price=100,
        volatility= Volatility(0.25),
        converse_rate=0.03
    )
rcki.compute_price(),rcki.compute_greeks()

(88.90331408025631,
 {'delta': 0.39113136920909136,
  'gamma': -0.015021300293950785,
  'theta': 3.2765250559945116,
  'rho': 0.47254376195503633,
  'vega': -0.37553250734876964})

In [3]:
from src.pricing.structured_products import OutperformerCertificate


outc = OutperformerCertificate(
        rate= Rate(0.03),
        maturity= Maturity(1),
        spot_price=100,
        strike_price=100,
        participation=10,
        volatility= Volatility(0.25),)
outc.compute_price(),outc.compute_greeks()

100


(202.1362914253212,
 {'delta': 6.370946058884719,
  'gamma': 0.13937288932531658,
  'theta': -56.60277734805595,
  'rho': 4.349583144631507,
  'vega': 3.4843222331329144})

# Pricing Options

## Vanilla Options


In [ ]:
from src.pricing.vanilla_options import VanillaOption


opt = VanillaOption(
    100,
    110,
    Maturity(maturity_in_days=365),
    Rate(rate=0.04),
    Volatility(volatility=0.20),
    option_type="call",
)

opt.compute_price()

In [ ]:
opt2 = VanillaOption(
    100,
    110,
    Maturity(maturity_in_days=365),
    Rate(rate=0.04),
    Volatility(volatility=0.20),
    option_type="put",
)

opt2.compute_price()

## Option digit


In [ ]:
from src.pricing.binary_options import BinaryOption


bin_opt = BinaryOption(
    100,
    110,
    Maturity(maturity_in_days=365),
    Rate(rate=0.04),
    Volatility(volatility=0.20),
    option_type="call",
)
bin_opt.compute_price()

In [ ]:
bin_opt2 = BinaryOption(
    100,
    110,
    Maturity(maturity_in_days=365),
    Rate(rate=0.04),
    Volatility(volatility=0.20),
    option_type="put",
)
bin_opt2.compute_price()

# Option strategies

## Straddle


In [ ]:
from src.pricing.option_strategies import StraddleStrategy


straddle = StraddleStrategy(
    100,
    140,
    Maturity(maturity_in_days=365),
    Rate(rate=0.04),
    Volatility(volatility=0.20),
)
straddle.compute_price()

In [ ]:
straddle.compute_greeks()

## Strangle strategy


In [ ]:
from src.pricing.option_strategies import StrangleStrategy


straddle = StrangleStrategy(
    100,
    90,
    110,
    Maturity(maturity_in_days=365),
    Rate(rate=0.04),
    Volatility(volatility=0.20),
)
straddle.compute_price()

# Pricing Bonds

## ZC


In [ ]:
from src.pricing.fixed_income import ZeroCouponBond


zc_bond = ZeroCouponBond(
    Rate(rate=0.03, rate_type="continuous"), Maturity(maturity_in_days=365), nominal=100
)


zc_bond.compute_price()

## Vanilla Bond

using a rate curve interpolation


In [ ]:
from src.pricing.fixed_income import Bond


rate_curve = {
    Maturity(1 / 365): 0.02,
    Maturity(1 / 12): 0.01,
    Maturity(3 / 12): 0.02,
    Maturity(6 / 12): 0.04,
    Maturity(1): 0.06,
    Maturity(3): 0.07,
    Maturity(5): 0.08,
    Maturity(10): 0.1,
}
bond = Bond(
    Rate(
        rate_curve=rate_curve,
        rate_type="continuous",
    ),
    Maturity(maturity_in_days=365 * 5),
    nominal=100,
    coupon_rate=0.02,
    nb_coupon=10,
)

bond.compute_price()

In [ ]:
from matplotlib import pyplot as plt


plt.plot(
    [mat.maturity_in_years for mat in rate_curve.keys()],
    [r for r in rate_curve.values()],
    label="Rate curve",
)
plt.xlabel("Tenors (Y)")
# plt.xscale('log')
plt.ylabel("Rate (%)")
plt.grid()
plt.legend()
plt.title("Rate curve")

## Barrier Options


In [ ]:
# Maturity en première clé et Strike en seconde
d1 = {
    "1.0": {
        "0.9": 0.14,
        "1.0": 0.10,
        "1.1": 0.12,
    },
    "1.5": {
        "0.9": 0.13,
        "1.0": 0.09,
        "1.1": 0.13,
    },
    "2.0": {
        "0.9": 0.10,
        "1.0": 0.1,
        "1.1": 0.08,
    },
}

In [ ]:
from itertools import product

volatility_surface_example = {}
for comb in product(map(float, d1.keys()), map(float, list(d1.items())[0][-1].keys())):
    volatility_surface_example[comb] = d1.get(str(comb[0])).get(
        str(comb[-1]), int(comb[-1])
    )

volatility_surface_example

In [ ]:
from src.pricing.base.volatility import Volatility


volatility_object = Volatility(volatility_surface=volatility_surface_example)

volatility_object.print_surface()

print(volatility_object.get_volatility(1.3, 0.75))

In [ ]:
from src.pricing.barrier_options import BarrierOption
import random
import numpy as np

random.seed(40)
np.random.seed(40)

barrier_option1 = BarrierOption(
    spot_price=80,
    strike_price=100,
    maturity=Maturity(maturity_in_days=365),
    rate=Rate(rate=0.04),
    volatility=Volatility(volatility=0.20),
    option_type="put",
    barrier_level=80,
    barrier_type="ki",
    barrier_direction="down",
)

price1 = barrier_option1.compute_price(num_paths=30000, num_steps=1000)
print(f"Le prix estimé de l'option barrière est: {price1}")

In [ ]:
barrier_option1.compute_greeks()